# Examples for finding optimum source configurations through basin hopping.

In [1]:
from SourceConfigurationOptimization.configOptimization import *

## Initializing the Configuration

First, we define the source type that we are working with. Suppose that firstly we would like to work with point sources at a distance from the capsule (radius a=1) of d=4.
We first require the a_l coefficients of the Legendre expansion, which fully defines the source type.

In [2]:
aLValues=aLValuesArray(4.84,1)

We then define an instance of the SourceType class

In [3]:
pointSource4=SourceType(aLValues)

We now wish to define our configuration. There are several ways to proceed; as an example, we will use 6 sources.

First, we may work with SingleSource objects.
We define the single sources by their polar coordinates and relative strength (theta, phi, strength).

In [4]:
numSources=6

In [5]:
s1=SingleSource(0,0,1)
s2=SingleSource(0.1,0,1)
s3=SingleSource(0.2,0,1)
s4=SingleSource(0.3,0,1)
s5=SingleSource(0.4,0,1)
s6=SingleSource(0.5,0,1)

Next we define our sourceArray, and then define the configuration object from the number of sources, SourceType, and array.

In [6]:
sourceArray=np.array([s1,s2,s3,s4,s5,s6])
config1=Configuration(numSources,pointSource4,sourceArray=sourceArray)

Alternatively, we may already have the information in the form of a configuration vector, which has all of the thetas, then phis, then strengths.
A further option is sourceTuples, in the form of a tuple (or anything list-like) of ((theta,phi,strength),...)

In [7]:
configVectorExample=np.array([0,0.1,0.2,0.3,0.4,0.5,0,0,0,0,0,0,1,1,1,1,1,1])
config2=Configuration(numSources,pointSource4,configVector=configVectorExample)

sourceTuplesExample=((0,0,1),(0.1,0,1),(0.2,0,1),(0.3,0,1),(0.4,0,1),(0.5,0,1))
config3=Configuration(numSources,pointSource4,sourceTuples=sourceTuplesExample)

In [8]:
print(config1.configVector,config2.configVector,config3.configVector )

[0.  0.1 0.2 0.3 0.4 0.5 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1. ] [0.  0.1 0.2 0.3 0.4 0.5 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1. ] [0.  0.1 0.2 0.3 0.4 0.5 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1. ]


Note that, to print with commas, you simple need to add repr()

In [9]:
print(repr(config1.configVector))

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 1. ,
       1. , 1. , 1. , 1. , 1. ])


Lastly, we can generate a random starting configuration by supplying no initial information.

In [10]:
config4=Configuration(numSources,pointSource4)
print(repr(config4.configVector))

array([2.26927718, 0.08200011, 0.03627205, 1.51813656, 2.69726907,
       3.05064266, 0.09245515, 5.8309031 , 4.67723544, 0.98526929,
       0.33835717, 4.79533172, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        ])


In [11]:
numLTerms=40
config1.sigmaRMSSqu(numLTerms)

2.6291118199684114

In [12]:
config1.sigmaRMSPercent(numLTerms)

162.1453613264472

## Optimizing the configuration

Now that we have our configuration, we define an instance of the Optimization class.
All settings are dealt with automatically - however, you will likely need to change the temp for good convergence. Aim to set the temp such that about 50% of basin hopping 'steps' are accepted.

In [13]:
optimizer=Optimization(config1,temp=0.0001)

To run numIterations steps of basin hopping:

In [14]:
numIterations=5
optimizer.optimize(numIterations)

basinhopping step 0: f 0.222399
basinhopping step 1: f 0.000244165 trial_f 0.000244165 accepted 1  lowest_f 0.000244165
found new global minimum on step 1 with function value 0.000244165
basinhopping step 2: f 0.000244167 trial_f 0.000244167 accepted 1  lowest_f 0.000244165
basinhopping step 3: f 0.000244167 trial_f 0.000244167 accepted 1  lowest_f 0.000244165
basinhopping step 4: f 0.000244171 trial_f 0.000244171 accepted 1  lowest_f 0.000244165
basinhopping step 5: f 0.000244166 trial_f 0.000244166 accepted 1  lowest_f 0.000244165


array([-3.14166838e+00, -1.57079759e+00, -4.42874414e-05,  1.57076528e+00,
        1.57071900e+00,  4.71234579e+00,  5.65530927e-01, -9.63981970e-01,
       -3.04221293e-03, -9.63993091e-01,  6.06850026e-01,  6.06784822e-01,
        1.36476395e+00,  1.36464141e+00,  1.36457286e+00,  1.36475595e+00,
        1.36467564e+00,  1.36467920e+00])

As it stands, this will update the initial configuration.

In [15]:
config1.configVector

array([-3.14166838e+00, -1.57079759e+00, -4.42874414e-05,  1.57076528e+00,
        1.57071900e+00,  4.71234579e+00,  5.65530927e-01, -9.63981970e-01,
       -3.04221293e-03, -9.63993091e-01,  6.06850026e-01,  6.06784822e-01,
        1.36476395e+00,  1.36464141e+00,  1.36457286e+00,  1.36475595e+00,
        1.36467564e+00,  1.36467920e+00])

In [16]:
config1.sigmaRMSPercent(numLTerms)

1.5625782582745884

Note that the first run will typically take longer, whilst numba compiles the functions.  

## Reduced Degrees of Freedom

For larger numbers of sources, we can speed up convergence by only dealing with half the sources - for reasons why see the paper.
In this case, we define our configuration using the ConfigurationRDOF class.

In [17]:
configVectorRDOF=np.array([0,0.1,0.2,0,0,0,1,1,1])
configRDOF=ConfigurationRDOF(numSources,pointSource4,configVector=configVectorRDOF)

To recover full configuration vector/source array/source tuples:

In [18]:
configRDOF.fullConfigVector()
configRDOF.fullSourceArray()
configRDOF.fullSourceTuples()

[(0.0, 0.0, 1.0),
 (0.1, 0.0, 1.0),
 (0.2, 0.0, 1.0),
 (3.141592653589793, 3.141592653589793, 1.0),
 (3.041592653589793, 3.141592653589793, 1.0),
 (2.941592653589793, 3.141592653589793, 1.0)]

Can be optimized exactly as before.

In [19]:
optimizerRDOF=Optimization(configRDOF,temp=0.0001)
numIterations=5
optimizerRDOF.optimize(numIterations)

basinhopping step 0: f 0.2224
basinhopping step 1: f 0.000244164 trial_f 0.000244164 accepted 1  lowest_f 0.000244164
found new global minimum on step 1 with function value 0.000244164
basinhopping step 2: f 0.000244164 trial_f 0.000244164 accepted 1  lowest_f 0.000244164
basinhopping step 3: f 0.000244164 trial_f 0.000244164 accepted 1  lowest_f 0.000244164
found new global minimum on step 3 with function value 0.000244164
basinhopping step 4: f 0.000244164 trial_f 0.000244164 accepted 1  lowest_f 0.000244164
basinhopping step 5: f 0.000244164 trial_f 0.000244164 accepted 1  lowest_f 0.000244164
found new global minimum on step 5 with function value 0.000244164


array([0.        , 1.57079401, 4.7123915 , 0.        , 0.        ,
       1.57079756, 1.        , 1.00000434, 0.99999957])

In [20]:
configRDOF.sigmaRMSPercent(numLTerms)

1.5625755267082277

## Studying Imperfections

First, define the standard deviations in angles (in radians) and intensities (as fraction of mean), alongside the number of perturbed configurations you would like in the ensemble.

In [21]:
stdTheta=0.035
stdPhi=0.035
stdInt=0.02
numConfigs=1000

Then use the stability test function, which returns the factor by which the perturbations increase sigma_rms from the initial, 'ideal' configuration, as well as the standard deviation in sigma_rms from these perturbations.

In [22]:
config1.stabilityTest(numLTerms,numConfigs,stdTheta,stdPhi,stdInt)

(2.9702498896430396, 0.010775458181017352)

## Lasers

For lasers instead of point sources, all is identical, apart from the SourceType.
Laser type is defined by a super-gaussian (see paper).
Need to give etaPerp, beta, and alpha.

In [23]:
etaPerp=0.9
beta=5
alpha=0.95

aLValuesLaser=aLValuesLaserArray(etaPerp,beta,alpha)
laser=SourceType(aLValuesLaser)

From this point, all is equivalent to the pointSources.